In [1]:
import pickle

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import gym

%matplotlib inline

In [2]:
sess = tf.InteractiveSession()

In [3]:
with open('./ant_train_test.pkl', 'rb') as inf:
    X_tv, y_tv, X_test, y_test = pickle.load(inf)

In [4]:
print(X_tv.shape, X_test.shape, y_tv.shape, y_test.shape)

(16008, 111) (4002, 111) (16008, 8) (4002, 8)


In [5]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[111])]

In [6]:
regressor = tf.estimator.DNNRegressor(
    feature_columns=feature_columns,
    hidden_units=[10, 20, 10],
    label_dimension=8,
#     model_dir="/tmp/iris_model"
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': 1, '_save_summary_steps': 100, '_log_step_count_steps': 100, '_model_dir': '/var/folders/79/dh9kb7m51m10qqhwqh4hg9pm0000gn/T/tmp27hoxf2g', '_save_checkpoints_secs': 600}


In [7]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_tv},
    y=y_tv,
    batch_size=128,
    num_epochs=5,
    shuffle=True)

In [8]:
regressor.train(input_fn=train_input_fn)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/79/dh9kb7m51m10qqhwqh4hg9pm0000gn/T/tmp27hoxf2g/model.ckpt.
INFO:tensorflow:step = 1, loss = 232.44
INFO:tensorflow:global_step/sec: 687.616
INFO:tensorflow:step = 101, loss = 20.0282 (0.147 sec)
INFO:tensorflow:global_step/sec: 727.256
INFO:tensorflow:step = 201, loss = 22.2819 (0.137 sec)
INFO:tensorflow:global_step/sec: 771.392
INFO:tensorflow:step = 301, loss = 17.2948 (0.130 sec)
INFO:tensorflow:global_step/sec: 746.307
INFO:tensorflow:step = 401, loss = 17.7119 (0.134 sec)
INFO:tensorflow:global_step/sec: 771.593
INFO:tensorflow:step = 501, loss = 17.5465 (0.130 sec)
INFO:tensorflow:global_step/sec: 777.375
INFO:tensorflow:step = 601, loss = 14.5475 (0.129 sec)
INFO:tensorflow:Saving checkpoints for 626 into /var/folders/79/dh9kb7m51m10qqhwqh4hg9pm0000gn/T/tmp27hoxf2g/model.ckpt.
INFO:tensorflow:Loss for final step: 6.00865.


In [9]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)

In [10]:
regressor.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Starting evaluation at 2017-10-20-04:18:50
INFO:tensorflow:Restoring parameters from /var/folders/79/dh9kb7m51m10qqhwqh4hg9pm0000gn/T/tmp27hoxf2g/model.ckpt-626
INFO:tensorflow:Finished evaluation at 2017-10-20-04:18:50
INFO:tensorflow:Saving dict for global step 626: average_loss = 0.016001, global_step = 626, loss = 16.009


{'average_loss': 0.016000958, 'global_step': 626, 'loss': 16.008959}

In [11]:
regressor.evaluate(input_fn=train_input_fn)

INFO:tensorflow:Starting evaluation at 2017-10-20-04:18:52
INFO:tensorflow:Restoring parameters from /var/folders/79/dh9kb7m51m10qqhwqh4hg9pm0000gn/T/tmp27hoxf2g/model.ckpt-626
INFO:tensorflow:Finished evaluation at 2017-10-20-04:18:53
INFO:tensorflow:Saving dict for global step 626: average_loss = 0.0153706, global_step = 626, loss = 15.7222


{'average_loss': 0.015370595, 'global_step': 626, 'loss': 15.722204}

In [42]:
def pred_action(model, obs):
    res = list(
        regressor.predict(
            tf.estimator.inputs.numpy_input_fn(
                x={'x': obs.reshape(1, -1)},
                shuffle=False)))
    return res[0]['predictions']

env = gym.make('Ant-v1')

obs = env.reset()
totalr = 0
done = False
max_timesteps = 600
for k in range(max_timesteps):
    if (k + 1) % 20 == 0:
        print(k + 1, end=',')
    action = pred_action(regressor, obs[None,:])
    obs, r, done, _ = env.step(action)
    totalr += r
    env.render()
env.render(close=True)
print()
print(totalr)
print(np.mean(totalr))

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'report_uninitialized_variables_1/boolean_mask/Gather:0' shape=(?,) dtype=string>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/Users/zyxue/Projects/sutton-barto-rl-exercises/berkeley-cs294/venv/lib/python3.5/runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/Users/zyxue/Projects/sutton-barto-rl-exercises/berkeley-cs294/venv/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/Users/zyxue/Projects/sutton-barto-rl-exercises/berkeley-cs294/venv/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/Users/zyxue/Projects/sutton-barto-rl-exercises/berkeley-cs294/venv/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 

INFO:tensorflow:Restoring parameters from /var/folders/79/dh9kb7m51m10qqhwqh4hg9pm0000gn/T/tmp27hoxf2g/model.ckpt-626
INFO:tensorflow:Restoring parameters from /var/folders/79/dh9kb7m51m10qqhwqh4hg9pm0000gn/T/tmp27hoxf2g/model.ckpt-626
INFO:tensorflow:Restoring parameters from /var/folders/79/dh9kb7m51m10qqhwqh4hg9pm0000gn/T/tmp27hoxf2g/model.ckpt-626
INFO:tensorflow:Restoring parameters from /var/folders/79/dh9kb7m51m10qqhwqh4hg9pm0000gn/T/tmp27hoxf2g/model.ckpt-626
INFO:tensorflow:Restoring parameters from /var/folders/79/dh9kb7m51m10qqhwqh4hg9pm0000gn/T/tmp27hoxf2g/model.ckpt-626
INFO:tensorflow:Restoring parameters from /var/folders/79/dh9kb7m51m10qqhwqh4hg9pm0000gn/T/tmp27hoxf2g/model.ckpt-626
INFO:tensorflow:Restoring parameters from /var/folders/79/dh9kb7m51m10qqhwqh4hg9pm0000gn/T/tmp27hoxf2g/model.ckpt-626
INFO:tensorflow:Restoring parameters from /var/folders/79/dh9kb7m51m10qqhwqh4hg9pm0000gn/T/tmp27hoxf2g/model.ckpt-626
INFO:tensorflow:Restoring parameters from /var/folders/7

KeyboardInterrupt: 

Bummer, seems estimator restore parameters from the file system everytime a prediction is done, very slow!

Seems estimator may not fit RL learning because RL is interactive. It's well fitted for predicting a batch of input features 